In [1]:
import logging

logging.basicConfig(format='[%(levelname)s@%(name)s] %(message)s', level=logging.DEBUG)

# Preparing data
## I use pandas for easy manipulation

In [3]:
import pandas as pd
import numpy as np
import datetime

# Change DataFrame dtypes to match Database models
def model_for_db(csv):
    """
    @parameters:
    - csv: Electronic Guide filename in comma separated values (.csv)
    
    @returns:
    - Dataframe with items formatted for bmat tv/av programs
    """
    
    # Load csv into Dataframe
    df = pd.read_csv(csv)

    # Fill n/a with readable string "n/a"
    df = df.fillna('n/a')
    df['program_year'] = df['program_year'].astype(str)
    df['start_time'] = (df['start_date'].astype(str)+' '+df['start_time']).map(lambda x: datetime.datetime.strptime(x, '%Y%m%d %H:%M'))
    df['duration_in_seconds'] = pd.to_timedelta(df['duration_in_seconds'], unit='seconds')
    df['end_time'] = df['start_time']+df['duration_in_seconds']
    df = df.drop(['start_date', 'duration_in_seconds'], axis=1) 
    return df


filename = 'epg.csv'


# Model for the db
df = model_for_db(filename)

if "__name__"=="__main__":
    import os
    import sys.argv
    
    filename = sys.argv[2]
    df = model_for_db(filename)
    

In [4]:
# What is the shape of the df
print("The data is {} cols by {} rows.\n".format(df.shape[1], df.shape[0]))
for column in df.columns:
    print(column, df[column].count())

print('\n'+'-'*31+'\n') 

print('The quantity of unique values in each col are the following:\n')
for column in df.columns:
    print(column, df[column].unique().size)
    
df.sample(3)

The data is 9 cols by 4897 rows.

('channel_id', 4897)
('channel_name', 4897)
('channel_country', 4897)
('start_time', 4897)
('program_id', 4897)
('program_original_title', 4897)
('program_local_title', 4897)
('program_year', 4897)
('end_time', 4897)

-------------------------------

The quantity of unique values in each col are the following:

('channel_id', 3)
('channel_name', 3)
('channel_country', 3)
('start_time', 4201)
('program_id', 1879)
('program_original_title', 358)
('program_local_title', 372)
('program_year', 44)
('end_time', 4147)


,channel_id,channel_name,channel_country,start_time,program_id,program_original_title,program_local_title,program_year,end_time
27,69036687,RTE 1,IRL,2018-01-01 10:15:00,5612095788,New Year's Day Concert 2018 With the Vienna Ph...,New Year's Day Concert 2018 With the Vienna Ph...,2018.0,2018-01-01 12:50:00
4482,69036190,La Sexta,ESP,2018-02-24 03:30:00,11742116,Minutos musicales,Minutos musicales,n/a,2018-02-24 05:00:00
760,69036687,RTE 1,IRL,2018-01-10 12:30:00,3180892269,n/a,The Shelbourne At Christmas,n/a,2018-01-10 13:00:00



<hr>

### *Observe there are cells in "program_original_title" and "program_year" that are null*


<hr>

# Add the df Showtimes to DB

In [6]:
import os
 
from django.db import IntegrityError
from tv import models

os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

# Get or create Program with channel obj and row data
def create_program(row, channel):
    if row.program_year != 'n/a':
        year = row.program_year[0:4]
    else:
        year = row.program_year
    program, created = models.Program.objects.get_or_create(uid=row.program_id,channel=channel,
                                                           year=year,
                                                           local_title=row.program_local_title,
                                                           original_title=row.program_original_title
                                                           )
    if created:
        program.save()
        print("Program {} with id {} added to DB.".format(program.local_title, 
                                                                program.uid,
                                                               ))
    return program

def create_channel(row):
    channel, created = models.Channel.objects.get_or_create(uid=row.channel_id, 
                                                            name=row.channel_name, 
                                                            country_code=row.channel_country
                                                           )
    if created:
        channel.save()
        print('Channel {} from {} with id {} added to DB.'.format(channel.name, 
                                                                  channel.country_code, 
                                                                  channel.id
                                                                 ))
    return channel

def create_showtime(row,program):
    showtime, created = models.Showtime.objects.get_or_create(program=program, 
                                                              start_time=row.start_time, 
                                                              end_time=row.end_time
                                                             )
    if created:
        showtime.save()
        print('Showtime starting on {} added to {}.'.format(row.start_time, 
                                                            program
                                                           ))
    return showtime

              
# Create Showtimes from DataFrame           
def add_showtimes_from_dataframe(df):
    for index, row in df.iterrows():
        channel = create_channel(row)
        program = create_program(row, channel)
        create_showtime(row, program)
            
    print("Finished adding programs from {}".format(filename))
        


In [7]:
import os

os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

from tv import models
for object in models.Program.objects.all():
    object.delete()
for object in models.Channel.objects.all():
    object.delete()


<hr>

# -v- Add programs to DB -v-

```python
addShowtimeFromDataFrame(df)
```

<hr>


In [8]:
add_showtimes_from_dataframe(df)

Channel HD1 (France) from FRA with id 38 added to DB.
Program À communiquer with id 461044 added to DB.
Showtime starting on 2018-01-01 00:00:00 added to Program: À communiquer with id 461044.
Channel La Sexta from ESP with id 39 added to DB.
Program Será anunciado with id 1887442 added to DB.
Showtime starting on 2018-01-01 00:00:00 added to Program: Será anunciado with id 1887442.
Channel RTE 1 from IRL with id 40 added to DB.
Program RTE 1 Programming with id 3452333967 added to DB.
Showtime starting on 2018-01-01 00:00:00 added to Program: RTE 1 Programming with id 3452333967.
Program L.A. Confidential with id 446381 added to DB.
Showtime starting on 2018-01-01 00:30:00 added to Program: L.A. Confidential with id 446381.
Program No controles with id 18063328 added to DB.
Showtime starting on 2018-01-01 00:45:00 added to Program: No controles with id 18063328.
Program Canal Ruleta with id 5072179538 added to DB.
Showtime starting on 2018-01-01 02:20:00 added to Program: Canal Ruleta

Program Avalanche with id 12116683 added to DB.
Showtime starting on 2018-01-01 20:00:00 added to Program: Avalanche with id 12116683.
Program La Sexta deportes with id 11922093 added to DB.
Showtime starting on 2018-01-01 20:05:00 added to Program: La Sexta deportes with id 11922093.
Program El intermedio Christmas Edition with id 5655111583 added to DB.
Showtime starting on 2018-01-01 20:30:00 added to Program: El intermedio Christmas Edition with id 5655111583.
Program RTÉ News: Nine O'Clock with id 12147320 added to DB.
Showtime starting on 2018-01-01 21:00:00 added to Program: RTÉ News: Nine O'Clock with id 12147320.
Program The Second Best Exotic Marigold Hotel with id 1189234487 added to DB.
Showtime starting on 2018-01-01 21:30:00 added to Program: The Second Best Exotic Marigold Hotel with id 1189234487.
Program Menace de glace with id 18231772 added to DB.
Showtime starting on 2018-01-01 21:30:00 added to Program: Menace de glace with id 18231772.
Program Cine with id 1198249

Showtime starting on 2018-01-02 19:50:00 added to Program: Petits plats en équilibre with id 12044548.
Program Estación laSexta with id 5599159988 added to DB.
Showtime starting on 2018-01-02 19:55:00 added to Program: Estación laSexta with id 5599159988.
Program Deux heures moins le quart avant Jésus-Christ with id 1154491 added to DB.
Showtime starting on 2018-01-02 20:00:00 added to Program: Deux heures moins le quart avant Jésus-Christ with id 1154491.
Program Fair City with id 5612079539 added to DB.
Showtime starting on 2018-01-02 20:00:00 added to Program: Fair City with id 5612079539.
Showtime starting on 2018-01-02 20:05:00 added to Program: La Sexta deportes with id 11922093.
Program El intermedio Christmas Edition with id 5668214450 added to DB.
Showtime starting on 2018-01-02 20:30:00 added to Program: El intermedio Christmas Edition with id 5668214450.
Showtime starting on 2018-01-02 20:30:00 added to Program: Senior Moments with id 4345613819.
Showtime starting on 2018-01

Program Neven's Food Trails: The Basque Country with id 4089228914 added to DB.
Showtime starting on 2018-01-03 19:30:00 added to Program: Neven's Food Trails: The Basque Country with id 4089228914.
Showtime starting on 2018-01-03 19:50:00 added to Program: Petits plats en équilibre with id 12044548.
Showtime starting on 2018-01-03 19:55:00 added to Program: Estación laSexta with id 5599159988.
Program Fair City with id 5612079523 added to DB.
Showtime starting on 2018-01-03 20:00:00 added to Program: Fair City with id 5612079523.
Program Section de recherches with id 66471005 added to DB.
Showtime starting on 2018-01-03 20:00:00 added to Program: Section de recherches with id 66471005.
Showtime starting on 2018-01-03 20:05:00 added to Program: La Sexta deportes with id 11922093.
Program EastEnders with id 5612084687 added to DB.
Showtime starting on 2018-01-03 20:30:00 added to Program: EastEnders with id 5612084687.
Program El intermedio Christmas Edition with id 5668214452 added to 

Showtime starting on 2018-01-04 13:05:00 added to Program: Urgences with id 12695644.
Showtime starting on 2018-01-04 13:25:00 added to Program: EastEnders with id 5612084687.
Showtime starting on 2018-01-04 13:50:00 added to Program: Urgences with id 13068078.
Showtime starting on 2018-01-04 13:55:00 added to Program: La Sexta Noticias: Jugones with id 4077761636.
Showtime starting on 2018-01-04 14:00:00 added to Program: Fair City with id 5612079523.
Showtime starting on 2018-01-04 14:20:00 added to Program: Jugones: El análisis with id 3286898281.
Showtime starting on 2018-01-04 14:30:00 added to Program: La sexta meteo with id 768381956.
Program Grand Designs with id 1190573309 added to DB.
Showtime starting on 2018-01-04 14:35:00 added to Program: Grand Designs with id 1190573309.
Showtime starting on 2018-01-04 14:45:00 added to Program: Urgences with id 12352545.
Program Zapeando with id 5668214472 added to DB.
Showtime starting on 2018-01-04 14:45:00 added to Program: Zapeando 

...

Showtime starting on 2018-02-01 09:45:00 added to Program: Au nom de la vérité with id 5706150785.
Program Al Rojo Vivo with id 5748722934 added to DB.
Showtime starting on 2018-02-01 10:00:00 added to Program: Al Rojo Vivo with id 5748722934.
Showtime starting on 2018-02-01 10:15:00 added to Program: Au nom de la vérité with id 5732814140.
Program Dr. Phil with id 4522798692 added to DB.
Showtime starting on 2018-02-01 10:20:00 added to Program: Dr. Phil with id 4522798692.
Showtime starting on 2018-02-01 10:50:00 added to Program: Au nom de la vérité with id 5685576509.
Showtime starting on 2018-02-01 11:15:00 added to Program: Au nom de la vérité with id 5654392726.
Program Shortland Street with id 5741755346 added to DB.
Showtime starting on 2018-02-01 11:15:00 added to Program: Shortland Street with id 5741755346.
Showtime starting on 2018-02-01 11:50:00 added to Program: Au nom de la vérité with id 5642731384.
Showtime starting on 2018-02-01 11:50:00 added to Program: Leaders' Qu

Showtime starting on 2018-02-02 07:10:00 added to Program: Petits secrets entre voisins with id 1147198869.
Showtime starting on 2018-02-02 07:20:00 added to Program: My Family with id 1329062.
Program Petits secrets entre voisins with id 1173541793 added to DB.
Showtime starting on 2018-02-02 07:40:00 added to Program: Petits secrets entre voisins with id 1173541793.
Showtime starting on 2018-02-02 07:55:00 added to Program: The Best of Today With Maura and Daithi with id 4113170753.
Showtime starting on 2018-02-02 08:10:00 added to Program: Au nom de la vérité with id 5647912989.
Program Crímenes imperfectos with id 17262805 added to DB.
Showtime starting on 2018-02-02 08:15:00 added to Program: Crímenes imperfectos with id 17262805.
Showtime starting on 2018-02-02 08:40:00 added to Program: Au nom de la vérité with id 5654395873.
Program Crímenes imperfectos with id 13821520 added to DB.
Showtime starting on 2018-02-02 08:45:00 added to Program: Crímenes imperfectos with id 13821520

Showtime starting on 2018-02-03 08:15:00 added to Program: Only Fools and Horses with id 349827.
Showtime starting on 2018-02-03 08:25:00 added to Program: Petits secrets entre voisins with id 4085282947.
Showtime starting on 2018-02-03 08:30:00 added to Program: Zapeando with id 5740316049.
Showtime starting on 2018-02-03 08:45:00 added to Program: Nationwide with id 5741755320.
Showtime starting on 2018-02-03 09:00:00 added to Program: Petits secrets entre voisins with id 4074450384.
Showtime starting on 2018-02-03 09:15:00 added to Program: Nationwide with id 5741755322.
Showtime starting on 2018-02-03 09:30:00 added to Program: Petits secrets entre voisins with id 1938005157.
Program Natural World with id 11790662 added to DB.
Showtime starting on 2018-02-03 09:45:00 added to Program: Natural World with id 11790662.
Showtime starting on 2018-02-03 10:00:00 added to Program: Petits secrets entre voisins with id 4074450316.
Showtime starting on 2018-02-03 10:15:00 added to Program: E

Showtime starting on 2018-02-04 06:00:00 added to Program: Euronews with id 12073476.
Showtime starting on 2018-02-04 06:20:00 added to Program: Petits secrets entre voisins with id 4098771660.
Program Piso compartido with id 1464743499 added to DB.
Showtime starting on 2018-02-04 06:35:00 added to Program: Piso compartido with id 1464743499.
Showtime starting on 2018-02-04 06:45:00 added to Program: Petits secrets entre voisins with id 4074450312.
Program Teleshopping with id 12185596 added to DB.
Showtime starting on 2018-02-04 06:55:00 added to Program: Teleshopping with id 12185596.


IntegrityError: FOREIGN KEY constraint failed

# -^- Programs added to DB -^-

<hr>


In [1]:
%%html
<style>
    table {float:left}
<style>

# TV API Reference

This API reference is organized by resource type. 
Each resource type has one or more data representations and one or more methods.

## Resource types
#### URIs relative to https://www.bmatapis.com/tv/v1, unless otherwise noted

 ### Channels
 
 | Method | HTTP request | Description |
 |:--|:--|:--|
 | list | **GET  /channels** | Lists the channels that broadcast programs. |


 ### Programs
 | Method | HTTP request | Description |
 |:--|:--|:--|
 | list | **GET  /programs** | Gets information about the programs broadcasted. **Required query parameters: startDate, endDate**  |


<hr>

# Channels: list

Lists the channels that broadcast programs.
### Request
#### HTTP request

**GET https://www.bmatapis.com/tv/v1/channels**

#### Parameters
| Parameter name | Value | Description |
|:--|:--|:--|
| Optional query parameters |
| id | integer | The ID of the channel |
| country | string | The A3 (UN) country code from which channels will be returned. |

#### Request body

**Do not supply a request body with this method.**

### Response

If successful, this method returns a response body with the following structure:

```python
{
  "id": integer,
  "name": string,
  "country": string,
  "country_code": string,
}
```

| Property name | Value | Description |  |
|:--|:--|:--|:--|
| id | integer | The channel id. |  |
| name | string | The name of the channel. |  |
| country | string | The country of the channel. |  |
| country_code | string | The A3 (UN) country code of the channel |  |
| programs | nested object | The list of programs aired on this channel. |  |


<hr>

# Programs: list

Lists the programs aired between dates with their broadcasting data (channel and
time details) aired within the specified time interval and country, and matching the specified title. 

### Request
#### HTTP request

**GET https://www.bmatapis.com/tv/v1/programs**

#### Parameters
| Parameter name | Value | Description |
|:--|:--|:--|
| **Required query parameters** |
| **startDate** | **string** | **The start date to search for programs from, formatted like: 'YYYYMMDD'** |
| **endDate** | **string** | **The end date to search for programs from, formatted like: 'YYYYMMDD'** |
| Optional query parameters |
| title | string | The local title of the program |
| country | string | The A3 (UN) country code from which channels will be returned. |


#### Request body

**Do not supply a request body with this method.**

### Response

If successful, this method returns a response body with the following structure:

```python
{
  "id": integer,
  "original_title": string,
  "local_title": string,
  "year": integer,
  "start_time": string,
  "end_time": string,
  "channel": {
    "id": integer,
    "name": string,
    "country": string,
    "country_code": string
  }
}
```

| Property name | Value | Description | Notes |
|:--|:--|:--|:--|
| id | integer | The id  of the file. |  |
| original_title | string | The original title of the program. |  |
| local_title | string | The local title of the program. |  |
| year | string | The year of the program. |  |
| start_time | string | The start time.  | Formatted as: **'YYYYmmdd - HH:MM'** |
| end_time | string | The end time. | Formatted as: **'YYYYmmdd - HH:MM'** |
| channel | nested object | The channel where the program was aired. |  |
| channel.id | integer | The channel id. |  |
| channel.name | string | The name of the channel where the program was aired. |  |
| channel.country | string | The country of the channel. |  |
| channel.country_code | string | The A3 (UN) country code of the channel |  |


<hr>

### Example response

```python
[
{
"program_id": "12044548",
"program_original_title": "Petits plats en équilibre","program_year": null,
"start_time": "20180101 - 13:00",
"end_time": "20180101 - 13:05",
...
},
{
"channel_id": "11939294",
"channel_name": "RTÉ News: One O'Clock",
"program_year": 2001,
"start_time": "20180101 - 13:00",
"end_time": "20180101 - 13:50",
},
...
]
```

In [199]:

tv_api_endpoint = 'https://www.bmatapis.com/tv/v1'
params = {
    start_date:'YYYYMMDD',
    end_date:'YYYYMMDD',
    country:'USA',
    title:'Du mmy tit le',
}
query_string = '?startDate=YYYYMMDD&endDate=YYYYMMDD&title=DU+MMY+TIT+LE&country=UN3'



NameError: name 'country' is not defined

In [200]:
channel = models.Channel.objects.all()[1]

In [201]:
channel.get_country_code_display()

'Spain'

In [202]:
program = channel.program_set.order_by('local_title','uid')

In [203]:
program

<QuerySet [<Program: Program: #laSexta Columna with id 22365312.>, <Program: Program: #laSexta Columna with id 5731404502.>, <Program: Program: #laSexta Columna with id 5752464022.>, <Program: Program: #laSexta Columna with id 5776045907.>, <Program: Program: #laSexta Columna with id 5799150496.>, <Program: Program: #laSexta Columna with id 5824127695.>, <Program: Program: Air Force One: Amenaza en el cielo with id 4042626150.>, <Program: Program: Al Rojo Vivo with id 21500310.>, <Program: Program: Al Rojo Vivo with id 5668214430.>, <Program: Program: Al Rojo Vivo with id 5668214432.>, <Program: Program: Al Rojo Vivo with id 5668214434.>, <Program: Program: Al Rojo Vivo with id 5668214438.>, <Program: Program: Al Rojo Vivo with id 5685798375.>, <Program: Program: Al Rojo Vivo with id 5685798389.>, <Program: Program: Al Rojo Vivo with id 5685798391.>, <Program: Program: Al Rojo Vivo with id 5685798393.>, <Program: Program: Al Rojo Vivo with id 5696564851.>, <Program: Program: Al Rojo Vi

In [204]:
program.showtime_set.order_by('-start_time')

AttributeError: 'QuerySet' object has no attribute 'showtime_set'

In [117]:
from django import forms

In [118]:
class ProgramForm(forms.Form):
    start_date = forms.DateInput()
    end_date = forms.DateField()

In [184]:
start_date = '2018-02-01'
end_date = '2018-02-02'
start_time = datetime.datetime.strptime(start_date, "%Y-%m-%d")
end_time = datetime.datetime.strptime(end_date, "%Y-%m-%d")
showtimes = Program.objects.filter(showtime__start_time__gt=start_time, showtime__end_time__lt=end_time).order_by('local_title','uid','-showtime__start_time')

In [185]:
showtimes.count()


79

In [186]:
showtimes

<QuerySet [<Program: Program: Al Rojo Vivo with id 5748722934.>, <Program: Program: Au nom de la vérité with id 5639938200.>, <Program: Program: Au nom de la vérité with id 5642731384.>, <Program: Program: Au nom de la vérité with id 5654392726.>, <Program: Program: Au nom de la vérité with id 5675873798.>, <Program: Program: Au nom de la vérité with id 5685576509.>, <Program: Program: Au nom de la vérité with id 5691609508.>, <Program: Program: Au nom de la vérité with id 5706150785.>, <Program: Program: Au nom de la vérité with id 5732814140.>, <Program: Program: Au nom de la vérité with id 5736556671.>, <Program: Program: Celebrity Operation Transformation with id 4036186478.>, <Program: Program: Comprando en casa with id 1076761458.>, <Program: Program: Crímenes imperfectos with id 13466420.>, <Program: Program: Crímenes imperfectos with id 17170252.>, <Program: Program: Doctor Foster with id 5285740607.>, <Program: Program: Dr. Phil with id 4522798692.>, <Program: Program: Ear to 

In [132]:
if local_title != '':
    print(True)